In [21]:
import os
import numpy as np
import matplotlib.pyplot as plt

from glob import glob
from astropy.table import Table
from automated_scripts.cal_uvis_make_darks import fileIO
from automated_scripts.cal_uvis_make_darks import dark_stats as ds

### Making the median dark current .dat files for unflashed and postflashed ref files

In [17]:
# first for unflashed superdarks
uf_loc = '/grp/hst/wfc3e/jmedina/uf_pl/superdarks'# location of the unflashed superdarks
superdarks = glob(os.path.join(uf_loc,'*'))
"""
expstarts = []
midpts_ext1 = []
midpts_ext4 = []

for superdark in superdarks:
    expstart, midpt_ext1, midpt_ext4 = ds.get_dark_median(superdark, 'superdarks')
    expstarts.append(expstart)
    midpts_ext1.append(midpt_ext1)
    midpts_ext4.append(midpt_ext4)
 
# putting into a table
t = Table([expstarts, midpts_ext1, midpts_ext4], names=('expstart', 'midpt_ext1', 'midpt_ext4'))
t.write('uf_midpt.dat', format='ascii')
""" 

In [79]:
from glob import glob
import logging
import os

from astropy.io import fits
import numpy as np

def calc_stats(stat_type, paths, anneal_date, ctecorr):
    """Builds a file within the `blvs` and `superdarks` stats subdirectories
    containing either dark current or hot pixel statistics, based on
    ``stat_type``.
    Creates 2 output statistic files (one under ``blvs`` and one under
    ``superdarks``), removing any already-existing ones, containing either dark
    current statistics (if ``stat_type`` is ``midpt``) or hot pixel statistics
    (if ``stat_type`` is ``hotpix``).
    Parameters
    ----------
    stat_type : str
        Can be either ``midpt`` to calculate dark current statistics
        or ``hotpix`` to calculate hot pixel statistics.
    paths : dict
        The dictionary containing the absolute paths of directories
        used throughout the pipeline.
    anneal_date : str
        The anneal date, in the format of ``YYYYMMDD``, to be used in
        the filename of the output file.
    ctecorr : bool
        ``True`` (if CTE-correction is turned on) or ``False`` (if
        CTE-correction is turned off). Used to determine the output
        filename.
    Notes
    -----
    Statistic files are replaced if they already exist so that the
    files are essentially updated, in case the pipeline is run multiple
    times throughout an anneal cycle.
    """

    # The process is performed twice:
    # 1) Using the blv_tmp.fits files to calculate the statistics
    # 2) Using the superdark files to calculate the statistics
    for data_type in ['superdarks']:

        # Build path to stat files we're about to create
        if ctecorr:
            path_ext1 = os.path.join(paths['plot_'+data_type], '{}/{}_stat_ext1_{}_ctecorr.dat'.format(stat_type, stat_type, anneal_date))
            path_ext4 = os.path.join(paths['plot_'+data_type], '{}/{}_stat_ext4_{}_ctecorr.dat'.format(stat_type, stat_type, anneal_date))
        else:
            path_ext1 = os.path.join(paths['plot_'+data_type], '{}/{}_stat_ext1_{}.dat'.format(stat_type, stat_type, anneal_date))
            path_ext4 = os.path.join(paths['plot_'+data_type], '{}/{}_stat_ext4_{}.dat'.format(stat_type, stat_type, anneal_date))

        # Remove stat files if they already exist
        if os.path.exists(path_ext1):
            os.remove(path_ext1)
        if os.path.exists(path_ext4):
            os.remove(path_ext4)

        # Calculate stats
        if stat_type == 'midpt':
            logging.info('\tCalculating dark median statistics.')
            logging.info('')
            logging.info('\t\tFilename\tEXPSTART\tDark median ext 1\tdark median ext4')
            files = glob(paths[data_type+'_for_dark'])
            for frame in files:
                expstart, stats_ext1, stats_ext4 = ds.get_dark_median(frame, data_type)
                filename = os.path.basename(frame)
                logging.info('\t\t{}\t{}\t{}\t{}'.format(filename, str(expstart), str(stats_ext1), str(stats_ext4)))
                ds.write_to_output(filename, expstart, stats_ext1, path_ext1)
                ds.write_to_output(filename, expstart, stats_ext4, path_ext4)
            logging.info('')
            logging.info('\tDark median statistic files written to {}'.format(os.path.join(paths['plot_'+data_type], 'midpt/')))
            logging.info('')

        elif stat_type == 'hotpix':
            logging.info('\tCalculating hot pixel statistics.')
            logging.info('')
            logging.info('\t\tFilename\tEXPSTART\tPercent hotpix ext 1\tPercent hotpix ext4')
            files = glob(paths[data_type+'_for_hotpix'])
            for frame in files:
                expstart, stats_ext1, stats_ext4 = ds.get_hotpix_stats(frame, data_type)
                filename = os.path.basename(frame)
                logging.info('\t\t{}\t{}\t{}\t{}'.format(filename, str(expstart), str(stats_ext1), str(stats_ext4)))
                ds.write_to_output(filename, expstart, stats_ext1, path_ext1)
                ds.write_to_output(filename, expstart, stats_ext4, path_ext4)
            logging.info('')
            logging.info('\tHot pixel statistic files written to {}'.format(os.path.join(paths['plot_'+data_type], 'hotpix/')))
            logging.info('')

In [73]:
for i in cycles_to_use:
    print(i.split('/')[-1].split('-')[-1].split('_')[0])

20090821
20090914
20091016
20091114
20091211
20100106
20100204
20100302
20100331
20100430
20100526
20100621
20100721
20100820
20100918
20101015
20101114
20101209
20110107
20110205
20110303
20110330
20110428
20110526
20110624
20110721
20110819
20110918
20111015
20111111
20111205
20120104
20120204
20120229
20120328
20120425
20120523
20120620
20120717
20120815
20120911
20121011
20121108
20121206
20130101
20130131
20130228
20130326
20130426
20130522
20130620
20130718
20130815
20130912
20131010
20131110
20131204
20140102
20140130
20140226
20140328
20140423
20140520
20140618
20140715
20140813
20140911
20141010
20141105
20141204
20141231
20150129
20150228
20150323
20150422
20150520
20150620
20150715
20150812
20150908
20151005
20151104
20151202
20151230
20151230
20160125
20160125
20160222
20160222
20160324
20160324
20160419
20160521
20160615
20160615
20160615
20160713
20160713
20160812
20160812
20160907
20160907
20161008
20161008
20161008
20161105
20161105
20161105
20161105
20161130
20161229
2

In [ ]:
# then for postflashed superdarks
# note: we are making the actual .dat files to put under /new_algorithm/superdarks/midpt
# directory for the quicklook darks monitor project
pf_loc = '/grp/hst/wfc3h/bourque/uvis_darks/new_algorithm/'# location of the
                                                           # postflash anneal cycles
anneal_cycles = glob(os.path.join(pf_loc,'post*'))
anneal_cycles = np.sort(anneal_cycles)



cycles_to_use = []
for anneal_cycle in anneal_cycles:   
    if 'ctecorr' not in anneal_cycle:
        cycles_to_use.append(anneal_cycle)

 
for cycle in cycles_to_use:
    anneal_date = cycle.split('/')[-1].split('-')[-1].split('_')[0]
    path = fileIO.make_path_dict(cycle)
    
    calc_stats('midpt', path, anneal_date, False)
    calc_stats('hotpix', path, anneal_date, False) 
"""
    pf_superdarks = glob(path['superdarks_for_dark']) # the superdarks delivered 
                                                      # for 'cycle'

    pf_expstarts = []
    pf_midpts_ext1 = []
    pf_midpts_ext4 = []
    
    for pf_superdark in pf_superdarks:
        pf_expstart, pf_midpt_ext1, pf_midpt_ext4 = ds.get_dark_median(pf_superdark, 'superdarks')
        pf_expstarts.append(expstart)
        pf_midpts_ext1.append(midpt_ext1)
        pf_midpts_ext4.append(midpt_ext4)
        
    t_ext1 = Table([os.path.basename(pf_superdark), 
                    expstart, 
                    midpts_ext4], 
                   names=('filename', 
                          'midpt_ext1', 
                          'midpt_ext1'))
    t.write('midpt_stat', format='ascii')
    """

In [86]:
for i in cycles_to_use:
    print(i)

/grp/hst/wfc3h/bourque/uvis_darks/new_algorithm/post-anneal-20090821_20150728
/grp/hst/wfc3h/bourque/uvis_darks/new_algorithm/post-anneal-20090914_20150729
/grp/hst/wfc3h/bourque/uvis_darks/new_algorithm/post-anneal-20091016_20150731
/grp/hst/wfc3h/bourque/uvis_darks/new_algorithm/post-anneal-20091114_20150801
/grp/hst/wfc3h/bourque/uvis_darks/new_algorithm/post-anneal-20091211_20150801
/grp/hst/wfc3h/bourque/uvis_darks/new_algorithm/post-anneal-20100106_20150816
/grp/hst/wfc3h/bourque/uvis_darks/new_algorithm/post-anneal-20100204_20150817
/grp/hst/wfc3h/bourque/uvis_darks/new_algorithm/post-anneal-20100302_20150820
/grp/hst/wfc3h/bourque/uvis_darks/new_algorithm/post-anneal-20100331_20150821
/grp/hst/wfc3h/bourque/uvis_darks/new_algorithm/post-anneal-20100430_20150821
/grp/hst/wfc3h/bourque/uvis_darks/new_algorithm/post-anneal-20100526_20150822
/grp/hst/wfc3h/bourque/uvis_darks/new_algorithm/post-anneal-20100621_20150823
/grp/hst/wfc3h/bourque/uvis_darks/new_algorithm/post-anneal-2010

In [62]:
path = fileIO.make_path_dict(cycles_to_use[-1])
ds.dark_stats_main(cycles_to_use[0]

'/grp/hst/wfc3h/bourque/uvis_darks/new_algorithm/post-anneal-20090821_20150728'

In [ ]:
# then for postflashed superdarks
# note: we are making the actual .dat files to put under /new_algorithm/superdarks/midpt
# directory for the quicklook darks monitor project

pf_loc = '/grp/hst/wfc3h/bourque/uvis_darks/new_algorithm/'# location of the
                                                           # postflash superdarks
directories = glob(os.path.join(pf_loc, 'post*'))
paths_to_use = []
for i in directories:
    if ('to' not in i) & ('ctecorr' not in i):
        paths_to_use.append(i)
pf_superdarks = []
for path in paths_to_use:
    superdarks = glob(os.path.join(main_dir,path,'deliver', '*drk*'))
    pf_superdarks.extend(superdarks)

pf_expstarts = []
pf_midpts_ext1 = []
pf_midpts_ext4 = []

for pf_superdark in pf_superdarks:
    expstart, midpt_ext1, midpt_ext4 = ds.get_dark_median(superdark, 'superdarks')
    pf_expstarts.append(expstart)
    pf_midpts_ext1.append(midpt_ext1)
    pf_midpts_ext4.append(midpt_ext4)

# Plotting

In [ ]:
%matplotlib qt

# adding sic&dh lockup lines
# adding shaded anneals 
# adding postflash start

# plotting data
fig, axs = plt.subplots(nrows=2,ncols=1,sharex=True)
# chip 1
axs[0].scatter(expstarts, midpts_ext4, s=20.0,color='coral', alpha=1, edgecolor='black',
            linewidth=0.3)
axs[0].scatter(pf_expstarts, pf_midpts_ext4, s=20.0,color='grey', alpha=1, edgecolor='black',
            linewidth=0.3)
# chip 2
axs[1].scatter(expstarts, midpts_ext1, s=20.0,color='coral', alpha=1, edgecolor='black',
            linewidth=0.3)
axs[1].scatter(pf_expstarts, pf_midpts_ext1, s=20.0,color='grey', alpha=1, edgecolor='black',
            linewidth=0.3)